In [0]:
dbutils.library.restartPython()

In [0]:
import sys

sys.path.append(
    "/Workspace/Users/bogdanovici.robert@gmail.com/opsfleet-project/src"
)

from run_pipeline import main

main()

## Q1 – What % of new users reach at least 30 seconds of watch_time in their first session?


In [0]:
spark.sql("""
SELECT
  SUM(
    CASE 
      WHEN session_number = 1 
           AND reached_30s = 1 
      THEN 1 
      ELSE 0 
    END
  ) * 100.0 
  / SUM(
      CASE 
        WHEN session_number = 1 
        THEN 1 
        ELSE 0 
      END
    ) AS percentage_users_reached_30_seconds_in_first_session,
  SUM(
    CASE 
      WHEN session_number = 1 
      THEN 1 
      ELSE 0 
    END
  ) AS number_of_users_with_first_session
FROM workspace.mart.fact_user_session
""").show()

## Q2 – Which video genres drive the highest 2nd-session retention within 3 days?

In [0]:
spark.sql("""
SELECT
  first_genre,
  AVG(
    CASE 
      WHEN session_number = 1 
      THEN retained_next_session_within_3d 
      ELSE NULL 
    END
  ) AS retention_second_session_within_3_days,
  COUNT(
    DISTINCT CASE 
      WHEN session_number = 1 
      THEN user_id 
      ELSE NULL 
    END
  ) AS number_of_users_in_genre
FROM workspace.mart.fact_user_session
WHERE first_genre IS NOT NULL
GROUP BY first_genre
ORDER BY retention_second_session_within_3_days DESC
""").show()

## Q3 – Is there a particular device_os or app_version where drop-off is abnormally high?

In [0]:
spark.sql("""
SELECT
  device_os,
  COUNT(DISTINCT user_id) AS number_of_users_in_device_os,
  AVG(
    CASE 
      WHEN session_number = 1 
      THEN dropoff_under_10s 
      ELSE NULL 
    END
  ) AS dropoff_rate_for_first_session
FROM workspace.mart.fact_user_session
GROUP BY device_os
ORDER BY dropoff_rate_for_first_session DESC
""").show()

In [0]:
import matplotlib.pyplot as plt

device_os_dropoff_df = spark.sql("""
SELECT
  device_os,
  COUNT(DISTINCT user_id) AS number_of_users_in_device_os,
  AVG(
    CASE 
      WHEN session_number = 1 
      THEN dropoff_under_10s 
      ELSE NULL 
    END
  ) AS dropoff_rate_for_first_session
FROM workspace.mart.fact_user_session
GROUP BY device_os
ORDER BY dropoff_rate_for_first_session DESC
""").toPandas()

plt.figure(figsize=(8, 5))
plt.bar(device_os_dropoff_df["device_os"], device_os_dropoff_df["dropoff_rate_for_first_session"])
plt.title("Drop-off rate (<10s) in first session by device_os")
plt.xlabel("device_os")
plt.ylabel("drop-off rate")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [0]:
spark.sql("""
SELECT
  app_version,
  COUNT(DISTINCT user_id) AS number_of_users_in_app_version,
  AVG(
    CASE 
      WHEN session_number = 1 
      THEN dropoff_under_10s 
      ELSE NULL 
    END
  ) AS dropoff_rate_for_first_session
FROM workspace.mart.fact_user_session
GROUP BY app_version
HAVING COUNT(DISTINCT user_id) >= 5
ORDER BY dropoff_rate_for_first_session DESC
""").show()

In [0]:
app_version_dropoff_df = spark.sql("""
SELECT
  app_version,
  COUNT(DISTINCT user_id) AS number_of_users_in_app_version,
  AVG(
    CASE 
      WHEN session_number = 1 
      THEN dropoff_under_10s 
      ELSE NULL 
    END
  ) AS dropoff_rate_for_first_session
FROM workspace.mart.fact_user_session
GROUP BY app_version
HAVING COUNT(DISTINCT user_id) >= 5
ORDER BY dropoff_rate_for_first_session DESC
""").toPandas()

plt.figure(figsize=(10, 10))
plt.barh(
    app_version_dropoff_df["app_version"],
    app_version_dropoff_df["dropoff_rate_for_first_session"]
)
plt.title("Drop-off rate (<10s) in first session by app_version (≥5 users)")
plt.xlabel("drop-off rate")
plt.ylabel("app_version")
plt.tight_layout()
plt.show()